In [ ]:
# choosing lemma because its important for interpreting adverse drug events accurately.
def lemmatizer(text):
    lemma = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    
    words = word_tokenize(text)

    lemmatized_words = [lemma.lemmatize(word) for word in words 
                        if word.lower() not in stop_words and word not in punctuation]

    lemmatized_text = ' '.join(lemmatized_words)

    return lemmatized_text

In [ ]:
negative_lemmatized_tokens = negative_df['context_clean'].apply(lemmatizer)
positive_lemmatized_tokens = positive_df['context_clean'].apply(lemmatizer)

In [ ]:
sentiment_mapping = {'Adverse_event': 1, 'Potential_therapeutic_event': 0}

In [ ]:
df['sentiment'] = df['event_type'].map(sentiment_mapping)

In [ ]:
df.sample(5)

In [ ]:
df.shape

In [ ]:
def preprocess_mesg(message):
    message = message.lower()
    message = re.sub(r'[^a-zA-Z0-9\s]', ' ', message)
    words = word_tokenize(message)
    lemmatizer = WordNetLemmatizer()
    stopwords_set = set(stopwords.words('english'))
    preprocessed_words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords_set]
    cleaned_message = " ".join(preprocessed_words)
    return cleaned_message

In [ ]:
# Load the data from CSV files
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
dev_data = pd.read_csv('data/dev.csv')

In [ ]:
sentiment_mapping = {'Adverse_event': 1, 'Potential_therapeutic_event': 0}

In [ ]:
train_data['sentiment'] = train_data['event_type'].map(sentiment_mapping)

In [ ]:
Split the data into features (X) and target (y)
X_train = train_data.drop('sentiment', axis=1)  # Replace 'target_column_name' with the actual name of your target column
y_train = train_data['sentiment']

# Split the test data into X_test and y_test
X_test = test_data.drop('sentiment', axis=1)
y_test = test_data['sentiment']

# Split the dev data into X_dev and y_dev
X_dev = dev_data.drop('sentiment', axis=1)
y_dev = dev_data['sentiment']

# Perform train-test split on the training data
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Print the shapes of the resulting datasets
print("Train set shape:", X_train_final.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)
print("Dev set shape:", X_dev.shape)


In [ ]:
X = df['context_clean'].values
y = df['sentiment'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)
y_pred = nb.predict(X_test_tfidf)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
formatted_accuracy = "{:.2%}".format(accuracy)
print("Accuracy:", formatted_accuracy)

In [ ]:
explainer = shap.Explainer(nb, X_train_tfidf)
shap_values = explainer(X_test_tfidf)
shap.summary_plot(shap_values, X_test_tfidf, feature_names=vectorizer.get_feature_names_out())

In [ ]:
def predict_sentiment(text):
    text_vectorized = vectorizer.transform([text])
    sentiment = classifier.predict(text_vectorized)[0]
    return sentiment

In [ ]:
text = "This drug is dangerous & creates high blood pressure."
sentiment = predict_sentiment(text)
print(sentiment)

In [ ]:
text = "This drug is very safe."
sentiment = predict_sentiment(text)
print(sentiment)